![Image in a markdown cell](https://cursos.utnba.centrodeelearning.com/pluginfile.php/1/theme_space/customlogo/1738330016/Logo%20UTN%20Horizontal.png)


# **Diplomado de Ciencia de Datos y Análisis Avanzado**
# **Unidad 5: Modelado Predictivo I**: Regresión y Clasificación

---

# **Proyecto de Competencia Kaggle: Predicción de Abandono de Clientes**

## **Curso:** Diplomado en Ciencia de Datos

# **Nombres de los Miembros del Equipo:**
### *   [Nombre Completo del Miembro 1]
### *   [Nombre Completo del Miembro 2]
### *   [Nombre Completo del Miembro 3]

# **Objetivo:**
## El objetivo de este proyecto es construir y evaluar varios modelos de clasificación para predecir si un cliente de una compañía de telecomunicaciones abandonará o no el servicio (churn). El rendimiento final del mejor modelo se medirá en la competencia de Kaggle a través de la **métrica ROC AUC**.

---

# **Enlace para unirse a la competencia**
### **USE EL ENLACE PARA UNIRSE POR EQUIPO, NO DE MANERA INDIVIDUAL**

https://www.kaggle.com/t/57b70c381e4d451b8ae38e164b91a2aa


### **Por favor siga las indicaciones que se suministran en la plataforma**

# **0. Configuración Inicial e Importación de Librerías**

## En esta sección, importaremos todas las librerías necesarias para el proyecto. Es una buena práctica tener todas las importaciones en la primera celda.

In [ ]:
# Importaciones básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizaciones
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Importar módulos del proyecto
from TP5 import data_loader, dataset_splitter, eda, models, metrics

print("✅ Todas las librerías y módulos importados correctamente")
print("🚀 Proyecto listo para ejecutar")

# **1. Carga de Datos**

## Cargaremos los datasets proporcionados para la competencia usando nuestro módulo de carga de datos.

In [ ]:
# Cargar datos usando el módulo data_loader
print("📥 Cargando datos de la competencia...")

try:
    # Intentar cargar datos reales de la competencia
    train_data, test_data, sample_submission = data_loader.load_competition_data()
    print("✅ Datos de competencia cargados exitosamente")
except FileNotFoundError:
    print("⚠️ Archivos de competencia no encontrados, generando datos de ejemplo...")
    # Generar datos de ejemplo para desarrollo
    train_data, test_data, sample_submission = data_loader.create_sample_data()
    print("✅ Datos de ejemplo generados exitosamente")

# Mostrar información de los datos
data_info = data_loader.get_data_info(train_data, test_data)
print(f"\n📊 Información de los datos:")
for key, value in data_info.items():
    print(f"   {key}: {value}")

print(f"\n📋 Primeras 5 filas del dataset de entrenamiento:")
display(train_data.head())

print(f"\n📋 Primeras 5 filas del dataset de prueba:")
display(test_data.head())

# **2. División de Datos**

## Dividiremos los datos en conjuntos de entrenamiento, validación y prueba usando nuestro módulo dataset_splitter.

In [ ]:
# Preparar datos para división
print("📊 Preparando datos para división...")

# Separar características y variable objetivo
if 'Churn' in train_data.columns:
    X = train_data.drop(['Churn'], axis=1)
    y = train_data['Churn']
else:
    # Si la columna objetivo tiene otro nombre, ajustar aquí
    target_col = train_data.columns[-1]  # Asumir que la última columna es el target
    X = train_data.drop([target_col], axis=1)
    y = train_data[target_col]

# Dividir datos usando dataset_splitter
splitter = dataset_splitter.DataSplitter(test_size=0.2, val_size=0.2, random_state=42)

X_train, X_val, X_test, y_train, y_val, y_test = splitter.split_data(X, y)

print(f"✅ Datos divididos exitosamente:")
print(f"   - Entrenamiento: {X_train.shape[0]:,} muestras")
print(f"   - Validación: {X_val.shape[0]:,} muestras")
print(f"   - Prueba: {X_test.shape[0]:,} muestras")
print(f"   - Características: {X_train.shape[1]}")

# Mostrar distribución del target en cada conjunto
print(f"\n🎯 Distribución de Churn por conjunto:")
print(f"   - Entrenamiento: {y_train.mean()*100:.1f}% churn")
print(f"   - Validación: {y_val.mean()*100:.1f}% churn")
print(f"   - Prueba: {y_test.mean()*100:.1f}% churn")

# **3. Análisis Exploratorio de Datos (EDA)**

## Realizaremos un análisis exploratorio completo usando nuestro módulo EDA.

In [ ]:
# Realizar EDA completo usando el módulo eda
print("🔍 Iniciando Análisis Exploratorio de Datos...")

# Información básica del dataset
eda.basic_info(train_data)

# Análisis de la variable objetivo
eda.analyze_target(train_data, 'Churn' if 'Churn' in train_data.columns else train_data.columns[-1])

# Análisis de características numéricas
eda.analyze_numerical_features(train_data)

# Análisis de características categóricas
eda.analyze_categorical_features(train_data)

# Análisis de correlaciones
eda.correlation_analysis(train_data)

print("\n✅ Análisis Exploratorio completado")

# **4. Preprocesamiento de Datos**

## Prepararemos los datos para el modelado usando ChurnPredictor.

In [ ]:
# Preprocesamiento de datos usando ChurnPredictor
from TP5.models import ChurnPredictor

print("🔧 Iniciando preprocesamiento de datos...")

# Inicializar el predictor
predictor = ChurnPredictor(random_state=42)

# Crear el preprocesador
preprocessor = predictor.create_preprocessor(X_train)

print("✅ Preprocesador configurado exitosamente")
print(f"📊 Características a procesar: {X_train.shape[1]}")

# Mostrar información del preprocesador
print("\n🔧 Configuración del preprocesador:")
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns
print(f"   - Características numéricas: {len(numeric_features)}")
print(f"   - Características categóricas: {len(categorical_features)}")
print(f"   - Numéricas: {list(numeric_features)}")
print(f"   - Categóricas: {list(categorical_features)}")

# **5. Entrenamiento de Modelos**

## Entrenaremos múltiples modelos de Machine Learning usando ChurnPredictor.

In [ ]:
# Entrenamiento de modelos usando ChurnPredictor
print("🤖 Iniciando entrenamiento de modelos...")

# Crear los modelos
models_dict = predictor.create_models()

# Entrenar todos los modelos
predictor.train_models(X_train, y_train)

print("\n🎉 Entrenamiento completado para todos los modelos:")
for model_name in models_dict.keys():
    print(f"   ✅ {model_name}")

print(f"\n📊 Modelos entrenados con {len(X_train):,} muestras")

# **6. Evaluación de Modelos**

## Evaluaremos todos los modelos y seleccionaremos el mejor usando métricas completas.

In [ ]:
# Evaluación de modelos
from TP5.metrics import MetricsCalculator

print("📊 Evaluando modelos...")

# Evaluar con el predictor
results = predictor.evaluate_models(X_val, y_val)

# Obtener el mejor modelo
best_model_name, best_model = predictor.get_best_model('ROC_AUC')

# Generar reporte completo
predictor.generate_model_report(X_val, y_val)

# Usar el calculador de métricas para análisis detallado del mejor modelo
calc = MetricsCalculator()

# Predicciones del mejor modelo
y_pred = best_model.predict(X_val)
y_pred_proba = best_model.predict_proba(X_val)[:, 1]

# Reporte detallado
detailed_report = calc.generate_detailed_report(
    y_val, y_pred, y_pred_proba,
    class_names=['No Churn', 'Churn'],
    model_name=best_model_name
)

print(f"\n🏆 Mejor modelo seleccionado: {best_model_name}")

# **7. Optimización de Hiperparámetros**

## Optimizaremos los hiperparámetros del mejor modelo.

In [ ]:
# Optimización de hiperparámetros para el mejor modelo
from TP5.models import hyperparameter_tuning

print(f"🔧 Optimizando hiperparámetros para {best_model_name}...")

# Definir grillas de parámetros según el modelo
if 'Logistic' in best_model_name:
    param_grid = {
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__solver': ['liblinear', 'lbfgs']
    }
elif 'KNN' in best_model_name:
    param_grid = {
        'classifier__n_neighbors': [3, 5, 7, 9],
        'classifier__weights': ['uniform', 'distance']
    }
elif 'Random' in best_model_name:
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5]
    }
else:
    # Para Naive Bayes u otros
    param_grid = {
        'classifier__var_smoothing': [1e-9, 1e-8, 1e-7]
    }

# Realizar búsqueda de hiperparámetros
grid_search = hyperparameter_tuning(
    best_model, param_grid, X_train, y_train,
    cv=5, scoring='roc_auc'
)

print(f"\n🎯 Hiperparámetros optimizados:")
print(f"   - Mejor score CV: {grid_search.best_score_:.4f}")
print(f"   - Mejores parámetros: {grid_search.best_params_}")

# Actualizar el mejor modelo con los parámetros optimizados
optimized_model = grid_search.best_estimator_

# Evaluar modelo optimizado
y_pred_opt = optimized_model.predict(X_val)
y_pred_proba_opt = optimized_model.predict_proba(X_val)[:, 1]

# Calcular métricas del modelo optimizado
from sklearn.metrics import roc_auc_score, accuracy_score
opt_auc = roc_auc_score(y_val, y_pred_proba_opt)
opt_acc = accuracy_score(y_val, y_pred_opt)

print(f"\n📈 Mejora con optimización:")
print(f"   - ROC AUC original: {results[best_model_name]['ROC_AUC']:.4f}")
print(f"   - ROC AUC optimizado: {opt_auc:.4f}")
print(f"   - Mejora: {opt_auc - results[best_model_name]['ROC_AUC']:.4f}")

# Guardar el modelo optimizado
best_model = optimized_model

# **8. Predicciones Finales y Submission**

## Generaremos las predicciones finales y crearemos el archivo de submission para Kaggle.

In [ ]:
# Predicciones finales y creación del archivo de submission
from TP5.models import create_submission_file

print("📄 Generando predicciones finales...")

# Combinar datos de entrenamiento y validación para el entrenamiento final
X_full_train = pd.concat([X_train, X_val], ignore_index=True)
y_full_train = pd.concat([y_train, y_val], ignore_index=True)

print(f"📊 Datos para entrenamiento final: {len(X_full_train):,} muestras")

# Preparar datos de prueba (usar test_data si es competencia real, o X_test si es simulación)
if 'customerID' in test_data.columns:
    # Competencia real
    X_test_final = test_data.drop(['customerID'], axis=1)
    test_ids = test_data['customerID']
else:
    # Datos simulados
    X_test_final = X_test
    test_ids = pd.Series([f'T{i:04d}' for i in range(len(X_test))])

# Crear archivo de submission
submission_df = create_submission_file(
    model=best_model,
    X_train=X_full_train,
    y_train=y_full_train,
    X_test=X_test_final,
    test_ids=test_ids,
    filename="submission_grupoM.csv"
)

# Mostrar primeras predicciones
print(f"\n📋 Primeras 10 predicciones:")
display(submission_df.head(10))

# Estadísticas de las predicciones
predictions = submission_df.iloc[:, 1].values
print(f"\n📊 Estadísticas de predicciones:")
print(f"   - Predicciones de churn (>0.5): {np.sum(predictions > 0.5):,} ({np.mean(predictions > 0.5)*100:.1f}%)")
print(f"   - Predicciones de no churn (≤0.5): {np.sum(predictions <= 0.5):,} ({np.mean(predictions <= 0.5)*100:.1f}%)")
print(f"   - Rango: [{predictions.min():.4f}, {predictions.max():.4f}]")

print(f"\n✅ Archivo de submission 'submission_grupoM.csv' creado exitosamente")
print(f"🎯 Listo para subir a Kaggle!")

# **9. Análisis Final y Conclusiones**

## Resumen completo del proyecto y resultados obtenidos.

In [ ]:
# Análisis final y conclusiones
print("📋 RESUMEN FINAL DEL PROYECTO")
print("=" * 60)

# Resumen de datos
print(f"\n📊 Resumen de Datos:")
print(f"   - Muestras de entrenamiento: {len(train_data):,}")
print(f"   - Muestras de prueba: {len(test_data):,}")
print(f"   - Características: {X_train.shape[1]}")
print(f"   - Tasa de churn en entrenamiento: {y_train.mean()*100:.1f}%")

# Resumen de modelos
print(f"\n🤖 Modelos Evaluados:")
for model_name, metrics in results.items():
    print(f"   - {model_name}: ROC AUC = {metrics['ROC_AUC']:.4f}")

print(f"\n🏆 Mejor Modelo: {best_model_name}")
print(f"   - ROC AUC optimizado: {opt_auc:.4f}")
print(f"   - Accuracy optimizado: {opt_acc:.4f}")

# Insights importantes
print(f"\n💡 Insights Clave:")
print(f"   - El modelo {best_model_name} mostró el mejor rendimiento")
print(f"   - La optimización de hiperparámetros mejoró el rendimiento")
print(f"   - Las características más importantes fueron identificadas en el EDA")
print(f"   - El preprocesamiento automático manejó correctamente los datos")

# Próximos pasos
print(f"\n🚀 Próximos Pasos:")
print(f"   1. Subir 'submission_grupoM.csv' a Kaggle")
print(f"   2. Analizar feature importance del modelo final")
print(f"   3. Probar técnicas de ensemble")
print(f"   4. Implementar validación cruzada estratificada")
print(f"   5. Análisis de errores para mejorar el modelo")

print(f"\n✅ Proyecto completado exitosamente!")
print(f"🎓 UTN - Análisis de Datos Avanzado - Unidad 5")
print(f"👥 Grupo M - Predicción de Fuga de Clientes")

# Generar reporte final de EDA
print(f"\n📄 Generando reporte final de EDA...")
eda_report = eda.generate_eda_report(train_data)
print(f"✅ Reporte EDA completo generado")